In [1]:
import tensorflow as tf
print(tf.__version__)
import cv2
print(cv2.__version__)
import time
import os
import numpy as np
import posenet
from collections import deque

c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0
3.4.5


# Function for calculating angle between two vectors

In [2]:

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return 180*(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))/np.pi

In [3]:
def findMin(q):
	if q[3] < q[0] and q[3] < q[1] and q[3] < q[2] and q[3] < q[4] and q[3] < q[5] and q[3] < q[6]:
		return q[3]
	else:
		return 0

# Parameters

In [13]:
path_input = './exercises'
path_output = './output'
print_min = True
model_number = 101
scale_factor = 0.2
delay = 4
exercise = "good_pullups.mp4"
body_points = {'rightShoulder':None,'rightWrist':None,'rightElbow':None,'leftShoulder':None,'leftWrist':None,'leftElbow':None}
body_points_score = {'rightShoulder':None,'rightWrist':None,'rightElbow':None,'leftShoulder':None,'leftWrist':None,'leftElbow':None}

# Image Analysis

In [ ]:
filenames = [f.path for f in os.scandir(path_input) if f.is_file() and f.path.endswith(('.png', '.jpg'))]

start = time.time()
with tf.Session() as sess:
    model_cfg, model_outputs = posenet.load_model(model_number, sess)
    output_stride = model_cfg['output_stride']
    for f in filenames:
        print(f)
        input_image, draw_image, output_scale = posenet.read_imgfile(
            f, scale_factor=scale_factor, output_stride=output_stride)
        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image}
        )

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.25)

        keypoint_coords *= output_scale


        draw_image = posenet.draw_skel_and_kp(
            draw_image, pose_scores, keypoint_scores, keypoint_coords,
            min_pose_score=0.25, min_part_score=0.25)

        cv2.imwrite(os.path.join(path_output, os.path.relpath(f, path_input)), draw_image)

        
        print("Results for image: %s" % f)
        for pi in range(len(pose_scores)):
            if pose_scores[pi] == 0.:
                break
            print('Pose #%d, score = %f' % (pi, pose_scores[pi]))
            for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                if posenet.PART_NAMES[ki] in body_points:
                    body_points[posenet.PART_NAMES[ki]] = c
                    body_points_score[posenet.PART_NAMES[ki]] = s
        #Calculate angle between shoulder and wrist using the elbow as the origin
        angle_right = angle((body_points['rightShoulder']-body_points['rightElbow']),(body_points['rightWrist']-body_points['rightElbow']))
        angle_left = angle((body_points['leftShoulder']-body_points['leftElbow']),(body_points['leftWrist']-body_points['leftElbow']))
        
        #Calculate the average confidence score of the shoulder, elbow, and wrist
        confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
        confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
        confidence_right /= 3
        confidence_left /= 3
        
        print('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
        print('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
        print()
    print('Average FPS:', len(filenames) / (time.time() - start))


# Video Analysis for Pushups and Pullups

In [15]:
with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(model_number, sess)
        output_stride = model_cfg['output_stride']
        cap = cv2.VideoCapture(exercise)
        cap.set(3, 1080)
        cap.set(4, 1920)
        #coordinate queue
        runningQ = deque()
        #create other queues to get more info
        moreInfoQ = deque()
        imageQ = deque()
        infoQ = deque()
        start = time.time()
        frame_count = 0
        picture_count = 1
        delay_count = 0
        export_data = []
        while True:
            res, img = cap.read()
            if not res:
                break
            else:
                input_image, display_image, output_scale = posenet.process_input(img, scale_factor, output_stride)
            info_dict = {"Body Points": None, "Body Points Score": None, "Scores Right": None, "Scores Left": None, "Image": None}
        
           
            #input_image, display_image, output_scale = posenet.read_cap(
              #  cap, scale_factor=0.2, output_stride=output_stride)
            
            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.15, min_part_score=0.1)
            
            overlay_image = cv2.resize(overlay_image, (1920, 1080))
            #overlay_image = cv2.resize(overlay_image, (600, 900))
            minimum_found = False

            imageQ.append(overlay_image)
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                #logging.warning('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    if posenet.PART_NAMES[ki] == "leftEye":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        runningQ.append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if len(runningQ) == 7:
                            if findMin(runningQ) != 0:
                                #print('FOUND MIN')
                                minimum_found = True
                                #print(findMin(runningQ))
                            runningQ.popleft()
                            #popleft to all other queues you create for more info here
                    if posenet.PART_NAMES[ki] in body_points:
                        body_points[posenet.PART_NAMES[ki]] = c
                        body_points_score[posenet.PART_NAMES[ki]] = s
                        
                            
                        
            info_dict['Body Points'] = body_points
            info_dict['Body Points Score'] = body_points_score
            
            #Calculate the average confidence score of the shoulder, elbow, and wrist
            confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
            confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
            confidence_right /= 3
            confidence_left /= 3
            info_dict['Scores Right'] = confidence_right
            info_dict['Scores Left'] = confidence_left
            info_dict['Image'] = overlay_image
            
            if delay_count < delay:
                infoQ.append(info_dict)
                delay_count+=1
            else:
                oldQ = infoQ.popleft()
                infoQ.append(info_dict)
            if print_min and minimum_found and confidence_right > 0.7 and confidence_left > 0.7:
                #Calculate angle between shoulder and wrist using the elbow as the origin
                angle_right = angle((oldQ['Body Points']['rightShoulder']-oldQ['Body Points']['rightElbow']),(oldQ['Body Points']['rightWrist']-oldQ['Body Points']['rightElbow']))
                angle_left = angle((oldQ['Body Points']['leftShoulder']-oldQ['Body Points']['leftElbow']),(oldQ['Body Points']['leftWrist']-oldQ['Body Points']['leftElbow']))
                print('For image number: ', picture_count)
                print('Maximum Height Found')
                print('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
                print('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
                print()
                cv2.imwrite(r'C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\output\maximum_pullup' + str(picture_count) + '.jpg',oldQ['Image'])
                export_data.append({'Maximum': picture_count, 'Angle Right':angle_right, 'Confidence Right': confidence_right, 'Angle Left':angle_left, 'Confidence Left': confidence_left})
                picture_count += 1
            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        print(export_data)
        cap.release()
        cv2.destroyAllWindows();
        print('Average FPS: ', frame_count / (time.time() - start))

C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\posenet\converter\config.py:9: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(cfg_f)


For image number:  1
Maximum Height Found
Angle Right: 75.641377 degrees with confidence 0.912146
Angle Left: 9.177983 degrees with confidence 0.834049

For image number:  2
Maximum Height Found
Angle Right: 26.745340 degrees with confidence 0.923074
Angle Left: 12.517180 degrees with confidence 0.941531

For image number:  3
Maximum Height Found
Angle Right: 38.938357 degrees with confidence 0.944993
Angle Left: 5.261306 degrees with confidence 0.781030

For image number:  4
Maximum Height Found
Angle Right: 20.776058 degrees with confidence 0.950561
Angle Left: 1.945642 degrees with confidence 0.815193

For image number:  5
Maximum Height Found
Angle Right: 54.967996 degrees with confidence 0.739111
Angle Left: 74.161132 degrees with confidence 0.862047

For image number:  6
Maximum Height Found
Angle Right: 145.361380 degrees with confidence 0.879148
Angle Left: 154.489382 degrees with confidence 0.807024

[{'Maximum': 1, 'Angle Right': 75.6413773039877, 'Confidence Right': 0.912145

# Video Analysis for Squat

In [ ]:
with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(model_number, sess)
        output_stride = model_cfg['output_stride']
        cap = cv2.VideoCapture(exercise)
        cap.set(3, 1080)
        cap.set(4, 1920)
        #coordinate queue
        runningQ = deque()
        #create other queues to get more info
        moreInfoQ = deque()
        imageQ = deque()
        infoQ = deque()
        start = time.time()
        frame_count = 0
        picture_count = 0
        delay_count = 0
        while True:
            res, img = cap.read()
            if not res:
                break
            else:
                input_image, display_image, output_scale = posenet.process_input(img, scale_factor, output_stride)
            info_dict = {"Body Points": None, "Body Points Score": None, "Scores Right": None, "Scores Left": None, "Image": None}
        
           
            #input_image, display_image, output_scale = posenet.read_cap(
              #  cap, scale_factor=0.2, output_stride=output_stride)
            
            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.15, min_part_score=0.1)
            
            #overlay_image = cv2.resize(overlay_image, (1024, 960))
            overlay_image = cv2.resize(overlay_image, (600, 900))
            minimum_found = False

            imageQ.append(overlay_image)
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                #logging.warning('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    if posenet.PART_NAMES[ki] == "leftEye":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        runningQ.append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if len(runningQ) == 7:
                            if findMin(runningQ) != 0:
                                #print('FOUND MIN')
                                minimum_found = True
                                #print(findMin(runningQ))
                            runningQ.popleft()
                            #popleft to all other queues you create for more info here
                    if posenet.PART_NAMES[ki] in body_points:
                        body_points[posenet.PART_NAMES[ki]] = c
                        body_points_score[posenet.PART_NAMES[ki]] = s
                        
                            
                        
            info_dict['Body Points'] = body_points
            info_dict['Body Points Score'] = body_points_score
            
            #Calculate the average confidence score of the shoulder, elbow, and wrist
            confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
            confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
            confidence_right /= 3
            confidence_left /= 3
            info_dict['Scores Right'] = confidence_right
            info_dict['Scores Left'] = confidence_left
            info_dict['Image'] = overlay_image
            
            if delay_count < delay:
                infoQ.append(info_dict)
                delay_count+=1
            else:
                oldQ = infoQ.popleft()
                infoQ.append(info_dict)
            if print_min and minimum_found:
                #Calculate angle between shoulder and wrist using the elbow as the origin
                angle_right = angle((oldQ['Body Points']['rightShoulder']-oldQ['Body Points']['rightElbow']),(oldQ['Body Points']['rightWrist']-oldQ['Body Points']['rightElbow']))
                angle_left = angle((oldQ['Body Points']['leftShoulder']-oldQ['Body Points']['leftElbow']),(oldQ['Body Points']['leftWrist']-oldQ['Body Points']['leftElbow']))
                print('Maximum Height Found')
                print('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
                print('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
                print()
                cv2.imwrite(r'C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\output\maximum_squat' + str(picture_count) + '.jpg',oldQ['Image'])

                picture_count += 1
            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows();
        print('Average FPS: ', frame_count / (time.time() - start))